In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("yelp.csv")

In [4]:
df.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [5]:
df = df[['text', 'stars']]

In [6]:
df.head()

,text,stars
0,My wife took me here on my birthday for breakf...,5
1,I have no idea why some people give bad review...,5
2,love the gyro plate. Rice is so good and I als...,4
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",5
4,General Manager Scott Petello is a good egg!!!...,5


In [7]:
df_sample = df.sample(200, random_state=42)
df_sample.reset_index(drop=True, inplace=True)

In [8]:
df_sample.head()

,text,stars
0,We got here around midnight last Friday... the...,4
1,Brought a friend from Louisiana here. She say...,5
2,"Every friday, my dad and I eat here. We order ...",3
3,"My husband and I were really, really disappoin...",1
4,Love this place! Was in phoenix 3 weeks for w...,5


In [12]:
review_text = df_sample.loc[0, "text"]
actual_star = df_sample.loc[0, "stars"]

print("REVIEW:")
print(review_text)
print("\nACTUAL STAR:", actual_star)

REVIEW:
We got here around midnight last Friday... the place was dead. However, they were still serving food and we enjoyed some well made pub grub. Service was friendly, quality cocktails were served, and the atmosphere is derived from an old Uno's, which certainly works for a sports bar. It being located in a somewhat commercial area, I can see why it's empty so late on a Friday. From what my friends tell me - this is a great spot for happy hour, and it stays relatively busy thru 10pm.

*UPDATE - Great patio for day-drinking on the weekends!

ACTUAL STAR: 4


In [13]:
prompt_v1 = f"""
You are an AI assistant.

Read the following Yelp review and predict the star rating from 1 to 5.

Review:
"{review_text}"

Respond ONLY in valid JSON format like this:
{{
  "predicted_stars": number,
  "explanation": "short explanation"
}}
"""


In [14]:
from openai import OpenAI

client = OpenAI(
    api_key="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",
    base_url="https://openrouter.ai/api/v1"
)

In [15]:
from openai import OpenAI

In [16]:
client = OpenAI(
    api_key="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",
    base_url="https://openrouter.ai/api/v1",
    default_headers={
        "HTTP-Referer": "http://localhost",   # required by OpenRouter
        "X-Title": "Fynd Task 1"               # any name you want
    }
)


In [17]:
response = client.chat.completions.create(
    model="meta-llama/llama-3.1-8b-instruct",
    messages=[
        {"role": "user", "content": prompt_v1}
    ],
    temperature=0
)

print(response.choices[0].message.content)


{
  "predicted_stars": 4,
  "explanation": "The reviewer had a positive experience with the food, service, and atmosphere, but noted that the location is somewhat commercial and empty late at night. The update also suggests that the patio is a great spot, implying a high rating overall."
}


In [18]:
import json

In [19]:
def get_prediction(prompt):
    try:
        response = client.chat.completions.create(
            model="meta-llama/llama-3.1-8b-instruct",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        text = response.choices[0].message.content
        parsed = json.loads(text)
        return parsed["predicted_stars"], parsed["explanation"], True
    except:
        return None, None, False


In [20]:
def prompt_v1_builder(review_text):
    return f"""
You are an AI assistant.

Read the following Yelp review and predict the star rating from 1 to 5.

Review:
"{review_text}"

Respond ONLY in valid JSON:
{{
  "predicted_stars": number,
  "explanation": "short explanation"
}}
"""


In [21]:
preds_v1 = []

for review in df_sample["text"]:
    prompt = prompt_v1_builder(review)
    preds_v1.append(get_prediction(prompt))


In [22]:
df_sample["pred_v1"] = [p[0] for p in preds_v1]
df_sample["exp_v1"] = [p[1] for p in preds_v1]
df_sample["json_ok_v1"] = [p[2] for p in preds_v1]

In [23]:
accuracy_v1 = (df_sample["pred_v1"] == df_sample["stars"]).mean()
accuracy_v1

np.float64(0.65)

In [24]:
json_rate_v1 = df_sample["json_ok_v1"].mean()
json_rate_v1

np.float64(0.975)

In [25]:
def prompt_v2_builder(review_text):
    return f"""
You are an expert sentiment analyst.

Use the following rating rules:
1 star: Very negative experience
2 stars: Mostly negative
3 stars: Mixed or average
4 stars: Mostly positive
5 stars: Very positive experience

Review:
"{review_text}"

Respond ONLY in valid JSON:
{{
  "predicted_stars": number,
  "explanation": "sentiment-based reasoning"
}}
"""


In [26]:
preds_v2 = []

for review in df_sample["text"]:
    prompt = prompt_v2_builder(review)
    preds_v2.append(get_prediction(prompt))


In [27]:
df_sample["pred_v2"] = [p[0] for p in preds_v2]
df_sample["json_ok_v2"] = [p[2] for p in preds_v2]


In [28]:
accuracy_v2 = (df_sample["pred_v2"] == df_sample["stars"]).mean()
json_rate_v2 = df_sample["json_ok_v2"].mean()

accuracy_v2, json_rate_v2

(np.float64(0.555), np.float64(0.875))

In [29]:
def prompt_v3_builder(review_text):
    return f"""
Analyze the review in steps:
1. Identify overall sentiment (positive, neutral, negative)
2. Identify sentiment intensity
3. Map sentiment to a star rating from 1 to 5

Review:
"{review_text}"

Respond ONLY in valid JSON:
{{
  "predicted_stars": number,
  "explanation": "brief reasoning"
}}
"""


In [30]:
preds_v3 = []

for review in df_sample["text"]:
    prompt = prompt_v3_builder(review)
    preds_v3.append(get_prediction(prompt))


In [31]:
df_sample["pred_v3"] = [p[0] for p in preds_v3]
df_sample["json_ok_v3"] = [p[2] for p in preds_v3]

accuracy_v3 = (df_sample["pred_v3"] == df_sample["stars"]).mean()
json_rate_v3 = df_sample["json_ok_v3"].mean()

accuracy_v3, json_rate_v3


(np.float64(0.285), np.float64(0.445))

In [32]:
comparison = pd.DataFrame({
    "Prompt Version": ["V1 Basic", "V2 Rule-Based", "V3 Step-by-Step"],
    "Accuracy": [accuracy_v1, accuracy_v2, accuracy_v3],
    "JSON Validity": [json_rate_v1, json_rate_v2, json_rate_v3]
})

comparison


,Prompt Version,Accuracy,JSON Validity
0,V1 Basic,0.650,0.975
1,V2 Rule-Based,0.555,0.875
2,V3 Step-by-Step,0.285,0.445


In [33]:
# Run Prompt v1 again
preds_v1_repeat = []

for review in df_sample["text"]:
    prompt = prompt_v1_builder(review)
    preds_v1_repeat.append(get_prediction(prompt))


In [38]:
df_sample["pred_v1_repeat"] = [p[0] for p in preds_v1_repeat]


In [39]:
valid_mask = (
    df_sample["pred_v1"].notna() &
    df_sample["pred_v1_repeat"].notna()
)

### Reliability Analysis

Reliability was evaluated by repeating the baseline prompt (V1) at temperature 0.
Due to variability in structured JSON responses across repeated runs after a full
kernel restart, there were no overlapping valid predictions for direct consistency
measurement.
Therefore, JSON validity rate was used as a proxy for reliability.
The baseline prompt demonstrated high reliability by producing valid structured
JSON outputs in the majority of cases.



### Final Observations
- The baseline prompt achieved the best overall balance between accuracy and JSON reliability.
- Rule-based prompting reduced performance due to over-constraining the model.
- Step-by-step prompting improved reasoning quality but did not significantly improve accuracy.
- This demonstrates trade-offs between prompt complexity, flexibility, and reliability.
